In [9]:
import os
import sys
import gzip
import time
import pickle
import subprocess
import pyfastx
import gzip
import pandas as pd
import multiprocessing 

In [3]:
df = pd.read_csv('/root/wangje/Project/Yin/Script/p5_barcode.txt', sep='\t')
df

,barcode1,barcode2
0,ACTTGATTGT,TCCTACCAGT
1,TAATCAGCTT,GCGTTGGAGC
2,TAAGAATGGT,GATCTTACGC
3,ACAACCTATT,CTGATGGTCA
4,ATTCCTAGAT,CCGAGAATCC
5,TTGATTCCTT,GCCGCAACGA
6,AAGGATTACT,TGAGTCTGGC
7,TCCTATAAGT,TGCGGACCTA
8,TATAAGAGGT,ACGGAGGCGG
9,TATTCTCCTT,TAGATCTACT


In [32]:
def Extract_UMI_Barcode(Read1):
    # 开始时间
    time_start = time.time()
    fa = pyfastx.Fastx(Read1)
    test_list = []
    all_seq_list = []
    pattern_seq = []
    one_barcode_list = []
    no_barcode_list = []
    tag = []
    for name, seq, comment in fa:
        all_seq_list.append(seq)
        len_all_seq_list = len(all_seq_list)
        if 'CAGAGC' in seq:
            pattern_seq.append(seq)
            len_pattern_seq = len(pattern_seq)
            # 一般情况下umi系列是靠近序列的5‘端，不需要考虑多匹配
            # print(seq.find('CAGAGC'))
            umi_index = int(seq.find('CAGAGC'))
            # print(seq[int(umi_index+6+8): int(umi_index+6+18)] == 'TCCTACCAGT')
            # 检测是否同时包含barcode1和barcode2
            # print(seq)
            # if df.loc[:,'barcode2'].str.contains(seq[int(umi_index+6+8): int(umi_index+6+18)]).any() and df.loc[:,'barcode1'].str.contains(seq[int(umi_index-10): int(umi_index)]).any():
            barcode1 = seq[int(umi_index-10):int(umi_index)]
            barcode2 = seq[int(umi_index+6+8): int(umi_index+6+18)]
            # print(barcode1)
            # print(barcode2)
            if barcode1 in df.loc[:, 'barcode1'].values and barcode2 in df.loc[:, 'barcode2'].values:
                test_list.append(seq)
                tag.append(name)
            elif barcode1 in df.loc[:, 'barcode1'].values or barcode2 in df.loc[:, 'barcode2'].values:
                one_barcode_list.append(seq)
            else:
                no_barcode_list.append(seq)
            # print(seq)
    time_end = time.time()
    time_sum = time_end - time_start
    print("""
          ---------------- tmp introduce ---------------------
          file: %s
          UMI---->>>
          UMI完全匹配序列数目: %s
          全部的序列数目: %s
          未匹配的序列数目: %s
          UMI+barcode---->>>
          两个barcode和UMI均匹配数目:%s
          其中一个barcode匹配的数目:%s
          barcode没有匹配的数目: %s
          耗时: %s s
          """ % (Read1, len_pattern_seq, len_all_seq_list, str(len_all_seq_list - len_pattern_seq), len(test_list), len(one_barcode_list), len(no_barcode_list), time_sum))

In [33]:
for file in os.listdir('/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/'):
    if file.endswith('1.fq.gz'):
        print(os.path.join('/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/',file))

/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-11_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-4_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-6_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-1_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-7_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-12_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-8_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-9_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-2_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-5_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-10_1.fq.gz
/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-3_1.fq.gz


In [34]:
Extract_UMI_Barcode(Read1='/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-1_1.fq.gz')


          ---------------- tmp introduce ---------------------
          file: /root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-1_1.fq.gz
          UMI---->>>
          UMI完全匹配序列数目: 1318927
          全部的序列数目: 1603731
          未匹配的序列数目: 284804
          UMI+barcode---->>>
          两个barcode和UMI均匹配数目:1207299
          其中一个barcode匹配的数目:93440
          barcode没有匹配的数目: 18188
          耗时: 285.4904754161835 s
          


In [35]:
for file1 in os.listdir('/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/'):
    if file1.endswith('1.fq.gz'):
        # print(os.path.join('/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/',file))
        sub_process = multiprocessing.Process(target=Extract_UMI_Barcode,kwargs={'Read1':f"{os.path.join('/root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院',file1)}"})
        sub_process.start()


          ---------------- tmp introduce ---------------------
          file: /root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-2_1.fq.gz
          UMI---->>>
          UMI完全匹配序列数目: 1076042
          全部的序列数目: 1291246
          未匹配的序列数目: 215204
          UMI+barcode---->>>
          两个barcode和UMI均匹配数目:985549
          其中一个barcode匹配的数目:76096
          barcode没有匹配的数目: 14397
          耗时: 56.560035705566406 s
          

          ---------------- tmp introduce ---------------------
          file: /root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J230304_J230304-6_1.fq.gz
          UMI---->>>
          UMI完全匹配序列数目: 1253324
          全部的序列数目: 1520471
          未匹配的序列数目: 267147
          UMI+barcode---->>>
          两个barcode和UMI均匹配数目:1146191
          其中一个barcode匹配的数目:89557
          barcode没有匹配的数目: 17576
          耗时: 65.0656156539917 s
          

          ---------------- tmp introduce ---------------------
          file: /root/wangje/Project/Yin/E150001524-中山大学孙逸仙纪念医院/J